In [ ]:
# Install dependencies
!pip install faiss-cpu transformers sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 23.5 MB/s eta 0:00:00


In [ ]:
# Import libraries
import os
import json
import pandas as pd
import faiss
import ast
import pickle
import numpy as np
from datasets import Dataset
from google.colab import files
from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [ ]:
# Load dataset from Google Drive
!gdown --id 1DKLke-54G6-s2_1ZKcO1jxOiEH7FpXqq

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1DKLke-54G6-s2_1ZKcO1jxOiEH7FpXqq
From (redirected): https://drive.google.com/uc?id=1DKLke-54G6-s2_1ZKcO1jxOiEH7FpXqq&confirm=t&uuid=badaa16a-fec6-4a4e-bf61-de3de6f3fd40
To: /content/dataset.csv
100% 2.29G/2.29G [00:29<00:00, 77.2MB/s]


In [ ]:
df = pd.read_csv('dataset.csv', delimiter=',', low_memory=False)
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
# Cleaning data
df = df.drop(columns=["Unnamed: 0", "source"])

def clean_list_column(col):
    def safe_parse(x):
        if isinstance(x, str):
            try:
                return " ".join(ast.literal_eval(x))
            except:
                return x
        return ""
    return col.apply(safe_parse)

df["ingredients"] = clean_list_column(df["ingredients"])
df["directions"] = clean_list_column(df["directions"])
df["NER"] = clean_list_column(df["NER"])

df["text"] = df["title"] + ". Ingredients: " + df["ingredients"] + ". Directions: " + df["directions"]

df = df[df["text"].notnull() & (df["text"].str.strip() != "")]

df = df.sample(n=20000, random_state=42).reset_index(drop=True)
df.to_pickle("cleaned_recipes.pkl")
df.head()

,title,ingredients,directions,link,NER,text
0,Ice Cream Krispies,12 cup butter 1 cup brown sugar 6 cups crisp r...,Prepare sauce: Cut strawberries in half. In a ...,www.food.com/recipe/ice-cream-krispies-223031,butter brown sugar crisp rice coconut nuts van...,Ice Cream Krispies. Ingredients: 12 cup butter...
1,Croatian Bean Stew With Smoked Pork Ribs,1 2/3 cups pinto beans 1 bay leaf 1 9/16 pound...,Cook beans together with a bay leaf in unsalte...,www.yummly.com/recipe/Croatian-Bean-Stew-With-...,pinto beans bay leaf pork celery root parsley ...,Croatian Bean Stew With Smoked Pork Ribs. Ingr...
2,Glazed Carrots,3 to 4 carrots 1 1/2 Tbsp. butter 1/3 c. brown...,Cook 3 to 4 carrots; cut crosswise in 1-inch p...,www.cookbooks.com/Recipe-Details.aspx?id=1011892,carrots butter brown sugar lemon rind,Glazed Carrots. Ingredients: 3 to 4 carrots 1 ...
3,Moms Pie Dough,4.5 Cups Flour 1.5 Tsp Salt Pinch Baking Powde...,Mix all dry ingredients in a bowl. Add crisco...,www.epicurious.com/recipes/member/views/moms-p...,Flour Salt Baking Powder Sugar Crisco egg vine...,Moms Pie Dough . Ingredients: 4.5 Cups Flour 1...
4,Pretzel Salad Or Dessert,2 c. crushed small thin pretzels (sticks) 3/4 ...,"Mix and press in baking pan, approximately 13 ...",www.cookbooks.com/Recipe-Details.aspx?id=106723,thin pretzels margarine,Pretzel Salad Or Dessert. Ingredients: 2 c. cr...


In [ ]:
# Embedding
model = SentenceTransformer("all-MiniLM-L6-v2")
model.save("./models/all-MiniLM-L6-v2")
corpus = df["text"].tolist()
embeddings = model.encode(corpus, convert_to_numpy=True, show_progress_bar=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

In [ ]:
# FAISS Index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [ ]:
faiss.write_index(index, "recipe_faiss.index")
with open("recipe_texts.pkl", "wb") as f:
    pickle.dump(corpus, f)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>